## MAI THỊ THẢO LY - 20280064

In [1]:
import pandas as pd
import numpy as np

# Input data

In [2]:
df=pd.read_csv('CSDL.csv')
df

,TID,Itemset
0,1,"Wine, Chips, Bread, Butter, Milk, Apple"
1,2,"Wine, Bread, Butter, Milk"
2,3,"Bread, Butter, Milk"
3,4,"Chips, Butter, Apple"
4,5,"Wine, Chips, Bread, Butter, Milk, Apple"
5,6,"Wine, Chips, Milk"
6,7,"Wine, Chips, Bread, Butter, Apple"
7,8,"Wine, Chips, Milk"
8,9,"Wine, Bread, Apple"
9,10,"Wine, Bread, Butter, Milk"


In [3]:
data=df['Itemset'].values

In [4]:
np.savetxt("temp.txt",data, fmt='%s')

# Split data into two sets, Itemset and TID

In [5]:
# open a file
Itemset = pd.read_csv("temp.txt", sep=', ',header=None)
Itemset

C:\Users\maith\AppData\Local\Temp\ipykernel_32704\324984109.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  Itemset = pd.read_csv("temp.txt", sep=', ',header=None)


,0,1,2,3,4,5
0,Wine,Chips,Bread,Butter,Milk,Apple
1,Wine,Bread,Butter,Milk,None,None
2,Bread,Butter,Milk,None,None,None
3,Chips,Butter,Apple,None,None,None
4,Wine,Chips,Bread,Butter,Milk,Apple
5,Wine,Chips,Milk,None,None,None
6,Wine,Chips,Bread,Butter,Apple,None
7,Wine,Chips,Milk,None,None,None
8,Wine,Bread,Apple,None,None,None
9,Wine,Bread,Butter,Milk,None,None


In [6]:
TID=df.iloc[:,:-1]
TID

,TID
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


# Vertical Apriori

In [7]:
def convert_to_vertical(TID,Itemset):
    Items = {}
    TID_list = {}
    
    Items_temp=np.unique(Itemset.astype(dtype='str').values.flatten())
    Items_temp=np.array(Items_temp)
    Items_temp=Items_temp[Items_temp!='None']
    
    for k in range(len(Items_temp)):
        Items[k]=Items_temp[k]
        TID_list[Items[k]]=[]
    TID_key=list(TID_list.keys())
    
    for l in range(len(TID_key)):
        for i in range(Itemset.shape[0]):
            for j in range(Itemset.shape[1]):
                if pd.isna(Itemset.iloc[i][j])==False:
                    if TID_key[l]==Itemset.iloc[i][j]:
                        TID_list[TID_key[l]].append(int(TID.values[i]))

    return Items,TID_list

In [8]:
def check_minsup(Items,TID_list,minsup):
    n=len(Items)
    Items_value=list(Items.values())
    Items_key=list(Items.keys())
    for i in range(n):
        if  len(Items)>0:
            if len(TID_list[str(Items_value[i])])< minsup :
                del TID_list[str(Items_value[i])]
                del Items[Items_key[i]]
        if len(Items)==0:
            return {},{}
    
    return Items,TID_list

In [9]:
def join_Itemset(Items):
    Items_key=list(Items.keys())
    Items_value=list(Items.values())
    Items_temp={}
    k=0
    
    for i in range(len(Items)):
        for j in range(i+1,len(Items)):
            Items_temp[k]=[]
            if str(Items_value[0]).count(',')==0:
                Items_temp[k].append(Items[Items_key[i]])
                Items_temp[k].append(Items[Items_key[j]])
            else:
                Items_temp[k]= [*set(Items[Items_key[i]]+Items[Items_key[j]])]
                
                if str(Items_temp[k]).count(',')!=str(Items_value[0]).count(',')+1:
                    del Items_temp[k]
            k=k+1
    for v in Items_temp.values():
        v.sort()
    result = {}
    for key,value in Items_temp.items():
        if value not in result.values():
            result[key] = value
    return result

In [10]:
def compare(arr1,arr2):
    arr=[]
    k=0
    for i in range(len(arr1)):
        for j in range(len(arr2)):
            if arr1[i]==arr2[j]:
                arr.append(arr1[i])
    return arr

In [11]:
def join_TID_list(Items,TID_list):
    Items_value=list(Items.values())
    TID_value=list(TID_list.values())
    TID_list_temp={}
    #temp1=[]
    for i in range(len(Items_value)):
        for j in range(len(Items_value[i])):
            for k in range(len(Items_value[i])):
                if k==0:
                    temp1=TID_list[Items_value[i][0]]
                else:
                    temp1=temp
                temp2=TID_list[Items_value[i][k]]
                temp=(compare(temp1,temp2))
                
            TID_list_temp[str(Items_value[i])]=temp
    return TID_list_temp

In [12]:
def VerticalApriori(TID,Itemset,minsup):
    F={}
    k=0
    Items,TID_list=convert_to_vertical(TID,Itemset)
    TID_list_temp=TID_list
    F[k]=TID_list
    
    while 1:
        Items=join_Itemset(Items)
        TID_list=join_TID_list(Items,TID_list_temp)
        Items,TID_list=check_minsup(Items,TID_list,minsup)
        if len(Items)>0:
            k=k+1
            F[k]=TID_list
        else:
            return F    
res=VerticalApriori(TID,Itemset,3)
res

{0: {'Apple': [1, 4, 5, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21],
  'Bread': [1, 2, 3, 5, 7, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22],
  'Butter': [1, 2, 3, 4, 5, 7, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20],
  'Chips': [1, 4, 5, 6, 7, 8, 11, 13, 16, 17, 18, 19, 21, 22],
  'Milk': [1, 2, 3, 5, 6, 8, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
  'Wine': [1, 2, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 19, 20, 21]},
 1: {"['Apple', 'Bread']": [1, 5, 7, 9, 11, 14, 15, 16, 17, 19, 20, 21],
  "['Apple', 'Butter']": [1, 4, 5, 7, 11, 12, 15, 16, 17, 18, 19, 20],
  "['Apple', 'Chips']": [1, 4, 5, 7, 11, 16, 17, 18, 19, 21],
  "['Apple', 'Milk']": [1, 5, 12, 14, 15, 16, 17, 18, 19, 20, 21],
  "['Apple', 'Wine']": [1, 5, 7, 9, 12, 14, 15, 16, 19, 20, 21],
  "['Bread', 'Butter']": [1, 2, 3, 5, 7, 10, 11, 13, 15, 16, 17, 19, 20],
  "['Bread', 'Chips']": [1, 5, 7, 11, 13, 16, 17, 19, 21, 22],
  "['Bread', 'Milk']": [1, 2, 3, 5, 10, 13, 14, 15, 16, 17, 19, 20, 21, 22],
  "['Bread', 'Wine'

# CHECK WITH ECLAT

In [13]:
# importing the ECLAT module
from pyECLAT import ECLAT
# loading transactions DataFrame to ECLAT class
eclat = ECLAT(data=Itemset)
# DataFrame of binary values
eclat.df_bin

,Bread,Milk,None,Wine,Butter,Apple,Chips
0,1,1,0,1,1,1,1
1,1,1,0,1,1,0,0
2,1,1,0,0,1,0,0
3,0,0,0,0,1,1,1
4,1,1,0,1,1,1,1
5,0,1,0,1,0,0,1
6,1,0,0,1,1,1,1
7,0,1,0,1,0,0,1
8,1,0,0,1,0,1,0
9,1,1,0,1,1,0,0


In [14]:
# count items in each row
items_per_transaction = eclat.df_bin.astype(int).sum(axis=1)
items_per_transaction

0     6
1     4
2     3
3     3
4     6
5     3
6     5
7     3
8     3
9     4
10    4
11    4
12    5
13    4
14    5
15    6
16    5
17    4
18    6
19    5
20    5
21    3
dtype: int64

In [15]:
# the item shoud appear at least at 5% of transactions
min_support = 2/22
# start from transactions containing at least 2 items
min_combination = 2
# up to maximum items per transaction
max_combination = max(items_per_transaction)
rule_indices, rule_supports = eclat.fit(min_support=min_support,
                                                 min_combination=min_combination,
                                                 max_combination=max_combination,
                                                 separator=' & ',
                                                 verbose=True)

Combination 2 by 2


15it [00:00, 172.96it/s]


Combination 3 by 3


20it [00:00, 145.85it/s]


Combination 4 by 4


15it [00:00, 128.93it/s]


Combination 5 by 5


6it [00:00, 116.74it/s]


Combination 6 by 6


1it [00:00, 57.91it/s]


In [16]:
result = pd.DataFrame(rule_supports.items(),columns=['Item', 'Support'])
result

,Item,Support
0,Bread & Milk,0.636364
1,Bread & Wine,0.590909
2,Bread & Butter,0.590909
3,Bread & Apple,0.545455
4,Bread & Chips,0.454545
5,Milk & Wine,0.636364
6,Milk & Butter,0.590909
7,Milk & Apple,0.500000
8,Milk & Chips,0.500000
9,Wine & Butter,0.500000
